In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import model_selection
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import nltk

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['Eluvio_DS_Challenge.csv']


In [ ]:
data = pd.read_csv("../input/Eluvio_DS_Challenge.csv")

In [46]:
over_18 = data[data["over_18"] == True]
under_18 = data[data["over_18"] == False].iloc[:100000]

In [47]:
over_18["t"] = over_18["title"].apply(lambda x: x.lower())
under_18["t"] = under_18["title"].apply(lambda x: x.lower())

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [50]:
from sklearn.feature_extraction.text import CountVectorizer

In [51]:
over_vectorizer = CountVectorizer(stop_words="english", max_df=.50, min_df=2)
over_vectors = over_vectorizer.fit_transform(over_18["title"].apply( \
    lambda x: x.lower()).values)

under_vectorizer = CountVectorizer(stop_words="english", max_df=.50, min_df=.0005)
under_vectors = under_vectorizer.fit_transform(under_18["title"].apply( \
    lambda x: x.lower()).values)

In [52]:
under_words = under_vectorizer.get_feature_names()
over_words = over_vectorizer.get_feature_names()
under_array = under_vectors.toarray()
over_array = over_vectors.toarray()

In [62]:
over_array = np.where(over_array >= 1.0, 1.0, 0)
under_array = np.where(under_array >= 1.0, 1.0, 0)

In [71]:
normalized_over_array = np.mean(over_array, axis=0)
normalized_under_array = np.mean(under_array, axis=0)

In [55]:
over_word_dict = {word: index for index, word in enumerate(over_words)}
under_word_dict = {word: index for index, word in enumerate(under_words)}

In [77]:
only_over_words = set(over_words) - set(under_words)
words_set = set(over_words) & set(under_words)

In [98]:
only_over_words

{'2013',
 'aleppo',
 'alight',
 'amateur',
 'amina',
 'bahraini',
 'bare',
 'bataclan',
 'behead',
 'beheading',
 'beheads',
 'blank',
 'bloodied',
 'boston',
 'breasts',
 'brutally',
 'bull',
 'canine',
 'cats',
 'corpse',
 'cruelty',
 'daraa',
 'dawn',
 'depicting',
 'describes',
 'disciplined',
 'disrupt',
 'disturbing',
 'drowned',
 'eid',
 'emerges',
 'execute',
 'executing',
 'express',
 'extremely',
 'femen',
 'feminist',
 'feminists',
 'golden',
 'goran',
 'gore',
 'halal',
 'horrific',
 'isil',
 'isis',
 'kiev',
 'lasts',
 'liberia',
 'mar',
 'marathon',
 'mela',
 'moments',
 'mosul',
 'nsfl',
 'nsfw',
 'nude',
 'original',
 'penis',
 'photographs',
 'pose',
 'posting',
 'session',
 'sniper',
 'snipers',
 'staged',
 'stray',
 'table',
 'tomasevic',
 'topless',
 'tragic',
 'ukrainian',
 'uncensored',
 'updated',
 'vagina',
 'vigilantes',
 'viral'}

In [87]:
word_scores = [
    (w, normalized_over_array[over_word_dict[w]], 
    normalized_under_array[under_word_dict[w]],
    )
    for w in words_set
]

In [90]:
scores = [
    (
        w,
        w1,
        w2,
        np.abs(w1 - w2)/(w1 + w2)
    ) for (w, w1, w2) in word_scores
]

In [96]:
sorted_scores = sorted(scores, key=lambda x: x[3], reverse=True)

In [97]:
# Top 100 words with greatest appearance difference
sorted_scores[:100]

[('graphic', 0.084375, 0.00094, 0.9779640157064995),
 ('protester', 0.028125, 0.00088, 0.9393208067574557),
 ('unarmed', 0.015625, 0.00059, 0.9272278754239901),
 ('naked', 0.021875, 0.00089, 0.9218097957390733),
 ('images', 0.0375, 0.00154, 0.9211065573770492),
 ('shocking', 0.01875, 0.00083, 0.9152196118488253),
 ('meat', 0.0125, 0.0006, 0.9083969465648856),
 ('footage', 0.0375, 0.00193, 0.90210499619579),
 ('execution', 0.01875, 0.00097, 0.9016227180527384),
 ('exclusive', 0.009375, 0.0005, 0.8987341772151898),
 ('truck', 0.0125, 0.00072, 0.8910741301059001),
 ('festival', 0.015625, 0.0009, 0.8910741301059001),
 ('photographer', 0.009375, 0.00055, 0.889168765743073),
 ('aftermath', 0.009375, 0.00055, 0.889168765743073),
 ('shoots', 0.009375, 0.00056, 0.887267237040765),
 ('damascus', 0.015625, 0.00094, 0.8865076969514035),
 ('paris', 0.025, 0.00154, 0.8839487565938207),
 ('neo', 0.0125, 0.00088, 0.8684603886397607),
 ('burned', 0.0125, 0.00092, 0.8628912071535021),
 ('videos', 0.0187